# K₇ Spectral Gap: High-Precision Extrapolation

## Strategy

Use validated methodology + Richardson extrapolation to estimate λ₁×H* at N→∞.

**Convergence theory** (7D manifold):
```
λ(N) = λ∞ + A × N^(-β)   where β ≈ 1/11 ≈ 0.091
```

**Method**:
1. Compute λ₁ at multiple N values (memory-optimized)
2. Fit power law to extract λ∞
3. Statistical confidence intervals

## Target: Determine if λ₁×H* = 13 exactly

In [ ]:
# Cell 1: Setup with Memory Management

import numpy as np
import json
import os
import gc
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats

# GPU support
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True
    print("✓ CuPy GPU available")
    
    # Memory info
    mempool = cp.get_default_memory_pool()
    print(f"  GPU memory: {mempool.total_bytes() / 1e9:.1f} GB allocated")
except ImportError:
    GPU = False
    from scipy.sparse import csr_matrix as cp_csr
    from scipy.sparse.linalg import eigsh as cp_eigsh
    cp = np
    print("CPU mode")

# Constants
H_STAR = 99
TCS_RATIO = H_STAR / 84
TARGET = 13  # dim(G₂) - 1
BETA_THEORY = 1/11  # Convergence exponent for 7D

def clear_memory():
    """Aggressive memory cleanup."""
    gc.collect()
    if GPU:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()

print(f"\nConstants:")
print(f"  H* = {H_STAR}")
print(f"  TCS ratio = {TCS_RATIO:.4f}")
print(f"  Target λ₁×H* = {TARGET}")
print(f"  Convergence β = {BETA_THEORY:.4f}")

In [ ]:
# Cell 2: Memory-Optimized Spectral Computation

def compute_lambda1_chunked(N, k, seed=42, chunk_size=5000):
    """Compute λ₁ with chunked distance computation for memory efficiency.
    
    For N=50k, full distance matrix = 50k² × 8 bytes = 20 GB
    With chunking, we compute k-NN distances in chunks.
    """
    np.random.seed(seed)
    
    # Sample K₇ points
    Q1 = np.random.randn(N, 4)
    Q1 /= np.linalg.norm(Q1, axis=1, keepdims=True)
    
    Q2 = np.random.randn(N, 4)
    Q2 /= np.linalg.norm(Q2, axis=1, keepdims=True)
    
    theta = np.random.uniform(0, 2*np.pi, N)
    
    # Build sparse W matrix via chunked k-NN
    print(f"    Building sparse graph (N={N}, k={k})...", end=" ", flush=True)
    
    # Storage for sparse matrix
    rows = []
    cols = []
    data = []
    
    # Global sigma estimation (sample)
    sample_idx = np.random.choice(N, min(1000, N), replace=False)
    D_sample = compute_distance_block(Q1, Q2, theta, sample_idx, sample_idx)
    knn_sample = np.partition(D_sample, min(k, len(sample_idx)-1), axis=1)[:, min(k, len(sample_idx)-1)]
    sigma = np.median(knn_sample)
    
    # Process in chunks
    n_chunks = (N + chunk_size - 1) // chunk_size
    
    for i in range(n_chunks):
        i_start = i * chunk_size
        i_end = min((i + 1) * chunk_size, N)
        idx_i = np.arange(i_start, i_end)
        
        # Compute distances to all points
        D_chunk = compute_distance_block(Q1, Q2, theta, idx_i, np.arange(N))
        
        # Find k nearest neighbors for each row
        for local_i, global_i in enumerate(idx_i):
            dists = D_chunk[local_i]
            # Get k nearest (excluding self)
            knn_idx = np.argpartition(dists, k+1)[:k+1]
            knn_idx = knn_idx[knn_idx != global_i][:k]
            
            # Gaussian weights
            weights = np.exp(-dists[knn_idx]**2 / (2 * sigma**2))
            
            rows.extend([global_i] * len(knn_idx))
            cols.extend(knn_idx.tolist())
            data.extend(weights.tolist())
        
        # Memory cleanup
        del D_chunk
        if i % 5 == 0:
            clear_memory()
    
    print("done.")
    
    # Build sparse W
    from scipy.sparse import coo_matrix, diags
    W = coo_matrix((data, (rows, cols)), shape=(N, N))
    W = W.tocsr()
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Normalized Laplacian
    degrees = np.array(W.sum(axis=1)).flatten()
    degrees = np.maximum(degrees, 1e-10)
    D_inv_sqrt = diags(1.0 / np.sqrt(degrees))
    
    L = diags(np.ones(N)) - D_inv_sqrt @ W @ D_inv_sqrt
    
    # Compute eigenvalues
    print(f"    Computing eigenvalues...", end=" ", flush=True)
    
    if GPU:
        L_gpu = cp_csr(cp.array(L.toarray()))
        eigs, _ = cp_eigsh(L_gpu, k=5, which='SA')
        eigs = cp.asnumpy(eigs)
    else:
        eigs, _ = cp_eigsh(L.tocsr(), k=5, which='SM')
    
    eigs = np.sort(eigs)
    lambda1 = eigs[1]  # First non-zero
    
    print(f"λ₁ = {lambda1:.6f}")
    
    # Cleanup
    del W, L, rows, cols, data
    clear_memory()
    
    return lambda1, sigma

def compute_distance_block(Q1, Q2, theta, idx_i, idx_j):
    """Compute geodesic distances between points idx_i and idx_j."""
    # S³ distances
    dot1 = np.abs(Q1[idx_i] @ Q1[idx_j].T)
    np.clip(dot1, 0, 1, out=dot1)
    d1 = 2.0 * np.arccos(dot1)
    
    dot2 = np.abs(Q2[idx_i] @ Q2[idx_j].T)
    np.clip(dot2, 0, 1, out=dot2)
    d2 = 2.0 * np.arccos(dot2)
    
    # S¹ distance
    dtheta = np.abs(theta[idx_i, None] - theta[None, idx_j])
    dtheta = np.minimum(dtheta, 2*np.pi - dtheta)
    
    # TCS metric
    D = np.sqrt(d1**2 + (TCS_RATIO**2) * d2**2 + dtheta**2)
    
    return D

print("Functions defined.")

In [ ]:
# Cell 3: Multi-N Convergence Study

print("="*60)
print("HIGH-PRECISION CONVERGENCE STUDY")
print("="*60)

# N values for extrapolation
# Memory: N² × 8 bytes (but chunked, so manageable)
N_values = [15000, 25000, 40000, 60000]

print(f"\nN values: {N_values}")
print(f"k = 0.74 × √N (optimal from research)")
print()

results = []

for N in N_values:
    k = int(0.74 * np.sqrt(N))
    print(f"\n--- N = {N:,}, k = {k} ---")
    
    # Multiple seeds for error estimation
    lambdas = []
    for seed in range(3):
        print(f"  Seed {seed}:")
        lambda1, sigma = compute_lambda1_chunked(N, k, seed=42+seed, chunk_size=3000)
        lambdas.append(lambda1)
    
    mean_lambda = np.mean(lambdas)
    std_lambda = np.std(lambdas)
    product = mean_lambda * H_STAR
    
    print(f"  → λ₁ = {mean_lambda:.6f} ± {std_lambda:.6f}")
    print(f"  → λ₁×H* = {product:.4f}")
    
    results.append({
        'N': N,
        'k': k,
        'lambda1_mean': float(mean_lambda),
        'lambda1_std': float(std_lambda),
        'product': float(product),
        'individual': [float(l) for l in lambdas]
    })
    
    clear_memory()

print("\n✓ Convergence study complete")

In [ ]:
# Cell 4: Richardson Extrapolation

print("="*60)
print("RICHARDSON EXTRAPOLATION")
print("="*60)

# Extract data
Ns = np.array([r['N'] for r in results])
products = np.array([r['product'] for r in results])
stds = np.array([r['lambda1_std'] * H_STAR for r in results])

# Model: λ×H* = λ∞×H* + A × N^(-β)
def convergence_model(N, lambda_inf, A, beta):
    return lambda_inf + A * np.power(N, -beta)

def convergence_model_fixed_beta(N, lambda_inf, A):
    return lambda_inf + A * np.power(N, -BETA_THEORY)

# Fit 1: Free β
try:
    popt_free, pcov_free = curve_fit(
        convergence_model, Ns, products,
        p0=[13, 100, 0.1],
        bounds=([0, -np.inf, 0.01], [20, np.inf, 0.5]),
        sigma=stds + 0.01,  # Add small constant for stability
        absolute_sigma=True
    )
    lambda_inf_free, A_free, beta_free = popt_free
    perr_free = np.sqrt(np.diag(pcov_free))
    fit_free_ok = True
except:
    fit_free_ok = False
    lambda_inf_free, beta_free = np.nan, np.nan

# Fit 2: Fixed β = 1/11 (theoretical)
try:
    popt_fixed, pcov_fixed = curve_fit(
        convergence_model_fixed_beta, Ns, products,
        p0=[13, 100],
        sigma=stds + 0.01,
        absolute_sigma=True
    )
    lambda_inf_fixed, A_fixed = popt_fixed
    perr_fixed = np.sqrt(np.diag(pcov_fixed))
    fit_fixed_ok = True
except:
    fit_fixed_ok = False
    lambda_inf_fixed = np.nan

# Simple linear extrapolation as backup
slope, intercept, r_value, p_value, std_err = stats.linregress(1/Ns, products)
lambda_inf_linear = intercept  # Value at 1/N → 0

print("\nExtrapolation Results:")
print("-" * 50)

if fit_free_ok:
    print(f"\n1. Free β fit:")
    print(f"   λ∞×H* = {lambda_inf_free:.4f} ± {perr_free[0]:.4f}")
    print(f"   β = {beta_free:.4f} (theory: {BETA_THEORY:.4f})")
    print(f"   Deviation from 13: {abs(lambda_inf_free-13)/13*100:.2f}%")

if fit_fixed_ok:
    print(f"\n2. Fixed β={BETA_THEORY:.4f} fit:")
    print(f"   λ∞×H* = {lambda_inf_fixed:.4f} ± {perr_fixed[0]:.4f}")
    print(f"   Deviation from 13: {abs(lambda_inf_fixed-13)/13*100:.2f}%")

print(f"\n3. Linear extrapolation (1/N → 0):")
print(f"   λ∞×H* = {lambda_inf_linear:.4f}")
print(f"   R² = {r_value**2:.4f}")
print(f"   Deviation from 13: {abs(lambda_inf_linear-13)/13*100:.2f}%")

# Best estimate
if fit_fixed_ok:
    best_estimate = lambda_inf_fixed
    best_error = perr_fixed[0]
    best_method = "Fixed β"
elif fit_free_ok:
    best_estimate = lambda_inf_free
    best_error = perr_free[0]
    best_method = "Free β"
else:
    best_estimate = lambda_inf_linear
    best_error = std_err * max(Ns)  # Rough estimate
    best_method = "Linear"

print(f"\n" + "="*50)
print(f"BEST ESTIMATE ({best_method}):")
print(f"  λ₁×H* (N→∞) = {best_estimate:.4f} ± {best_error:.4f}")
print(f"  Target = 13")
print(f"  Deviation = {abs(best_estimate-13)/13*100:.2f}%")
print(f"="*50)

In [ ]:
# Cell 5: Visualization and Save

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Raw convergence
ax1 = axes[0, 0]
ax1.errorbar(Ns, products, yerr=stds, fmt='bo-', linewidth=2, markersize=8, capsize=5, label='Measured')
ax1.axhline(y=13, color='g', linestyle='--', linewidth=2, label='Target 13')
ax1.axhline(y=14, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Pell 14')
ax1.set_xlabel('N (sample size)', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Convergence with Sample Size', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Extrapolation fit
ax2 = axes[0, 1]
N_plot = np.linspace(min(Ns)*0.5, max(Ns)*2, 100)

ax2.errorbar(Ns, products, yerr=stds, fmt='bo', markersize=10, capsize=5, label='Data')

if fit_fixed_ok:
    ax2.plot(N_plot, convergence_model_fixed_beta(N_plot, *popt_fixed), 'r-', 
             linewidth=2, label=f'Fit: λ∞×H* = {lambda_inf_fixed:.2f}')

ax2.axhline(y=13, color='g', linestyle='--', linewidth=2, alpha=0.7)
ax2.axhline(y=best_estimate, color='red', linestyle=':', linewidth=2, label=f'Extrapolated: {best_estimate:.2f}')

ax2.set_xlabel('N', fontsize=12)
ax2.set_ylabel('λ₁ × H*', fontsize=12)
ax2.set_title('Richardson Extrapolation', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: 1/N plot (linear extrapolation)
ax3 = axes[1, 0]
inv_N = 1 / Ns
ax3.errorbar(inv_N, products, yerr=stds, fmt='bo', markersize=10, capsize=5)
ax3.plot([0, max(inv_N)*1.2], [intercept, intercept + slope*max(inv_N)*1.2], 'r-', linewidth=2)
ax3.axhline(y=13, color='g', linestyle='--', linewidth=2, alpha=0.7)
ax3.scatter([0], [intercept], color='red', s=100, zorder=5, marker='*', label=f'N→∞: {intercept:.2f}')
ax3.set_xlabel('1/N', fontsize=12)
ax3.set_ylabel('λ₁ × H*', fontsize=12)
ax3.set_title(f'Linear Extrapolation (R² = {r_value**2:.3f})', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xlim(-0.00001, max(inv_N)*1.2)

# Plot 4: Final result
ax4 = axes[1, 1]
categories = ['Measured\n(best N)', 'Extrapolated\n(N→∞)', 'Target\n13', 'Pell\n14']
values = [products[-1], best_estimate, 13, 14]
colors = ['steelblue', 'red', 'green', 'orange']
bars = ax4.bar(categories, values, color=colors, alpha=0.7)

# Error bar on extrapolated
ax4.errorbar(1, best_estimate, yerr=best_error, fmt='none', color='black', capsize=10, linewidth=2)

for bar, val in zip(bars, values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
             f'{val:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax4.set_ylabel('λ₁ × H*', fontsize=12)
ax4.set_title('Final Comparison', fontsize=12)
ax4.set_ylim(0, 16)

plt.tight_layout()
os.makedirs('outputs', exist_ok=True)
plt.savefig('outputs/k7_spectral_precision.png', dpi=150, bbox_inches='tight')
plt.show()

# Save results
final_results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'K7_Spectral_Precision.ipynb',
        'method': 'Richardson extrapolation with chunked graph Laplacian'
    },
    'convergence_data': results,
    'extrapolation': {
        'free_beta': {
            'lambda_inf': float(lambda_inf_free) if fit_free_ok else None,
            'beta': float(beta_free) if fit_free_ok else None
        },
        'fixed_beta': {
            'lambda_inf': float(lambda_inf_fixed) if fit_fixed_ok else None,
            'beta_theory': float(BETA_THEORY)
        },
        'linear': {
            'lambda_inf': float(lambda_inf_linear),
            'r_squared': float(r_value**2)
        }
    },
    'final_result': {
        'lambda1_times_Hstar': float(best_estimate),
        'uncertainty': float(best_error),
        'method': best_method,
        'deviation_from_13_pct': float(abs(best_estimate-13)/13*100),
        'consistent_with_13': bool(abs(best_estimate-13) < 2*best_error + 0.5)
    }
}

with open('outputs/k7_spectral_precision.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("\nSaved: outputs/k7_spectral_precision.png")
print("Saved: outputs/k7_spectral_precision.json")

print("\n" + "="*60)
print("FINAL CONCLUSION")
print("="*60)
print(f"\n  λ₁ × H* (N→∞) = {best_estimate:.3f} ± {best_error:.3f}")
print(f"\n  Target (dim G₂ - 1) = 13")
print(f"  Deviation = {abs(best_estimate-13)/13*100:.2f}%")
print(f"\n  Consistent with 13: {final_results['final_result']['consistent_with_13']}")